# INTRODUCTION

Welcome to the **Research Trach II** first assignment! This Jupyter notebook provides an interactive interface for Research Track I second assignment to monitor the position and velocity of a robot, set and cancel goals, and visualize the performance using interactive plots and widgets. Utilize ROS for real-time monitoring and goal management, and `ipywidgets` for an intuitive user experience.

**Developer:** Seyed Alireza Mortazavi

**Student Number:** S6136275

# Import Libraries
We start by importing the necessary libraries.

In [ ]:
# ROS imports for robot communication and message handling
import rospy
from nav_msgs.msg import Odometry
import actionlib
from assignment_2_2023.msg import Vel, PlanningAction, PlanningGoal

# IPython widgets for creating interactive user interfaces in Jupyter notebooks
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Matplotlib imports for plotting
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Additional imports for transformations and numerical operations
import tf
from tf.transformations import quaternion_matrix
import numpy as np

# Tkinter for embedding Matplotlib plots in Tkinter windows (if needed)
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


# ROS Components Initialization

The `ros_components_init` function initializes essential ROS components required for the robot's operation, including the action client and publisher.

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px;">
    <h3>Function: ros_components_init</h3>
    <ul>
        <li><strong>Initialize ROS Node:</strong> The ROS node is initialized with a unique name using <code>rospy.init_node</code>. The <code>anonymous=True</code> parameter ensures the node has a unique name by adding random numbers to the end of the node's name.</li>
        <li><strong>Action Client:</strong> An action client for the <code>reaching_goal</code> action is created using <code>actionlib.SimpleActionClient</code>. This client will be used to send goals to the robot.</li>
        <li><strong>Wait for Server:</strong> The client waits until the action server is ready to receive goals using <code>client.wait_for_server</code>.</li>
        <li><strong>Goal Cancelled Flag:</strong> A boolean flag <code>goal_cancelled</code> is initialized to <code>False</code> to track whether a goal has been cancelled.</li>
        <li><strong>Publisher:</strong> A publisher is created using <code>rospy.Publisher</code> to publish position and velocity information to the <code>/pos_vel</code> topic.</li>
        <li><strong>Return Values:</strong> The function returns the publisher, client, and <code>goal_cancelled</code> flag.</li>
    </ul>
</div>

In [ ]:
def ros_components_init():
    global client 
    rospy.init_node('set_target_client', anonymous=True)  
    client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction) 
    client.wait_for_server()
    goal_cancelled = False
    pub = rospy.Publisher("/pos_vel", Vel, queue_size=1) 
    return pub, client, goal_cancelled


# User Interface Initialization

The `init_widgets` function sets up the interactive user interface elements, including labels, input fields, buttons, and a dropdown menu, using `ipywidgets`. The `update_labels` function updates the labels for position and velocity.

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px;">
    <h3>Function: init_widgets</h3>
    <ul>
        <li><strong>Global Variables:</strong> <code>goal_status</code> and <code>plot_selector</code> are declared as global to be accessible throughout the program.</li>
        <li><strong>Labels:</strong> 
            <ul>
                <li><code>pos_label</code>: Displays the current position.</li>
                <li><code>vel_label</code>: Displays the current velocity.</li>
                <li><code>target_label</code>: Displays the target coordinates.</li>
            </ul>
        </li>
        <li><strong>Input Fields:</strong> 
            <ul>
                <li><code>goal_x_input</code>: Input field for the x-coordinate of the goal.</li>
                <li><code>goal_y_input</code>: Input field for the y-coordinate of the goal.</li>
            </ul>
        </li>
        <li><strong>Buttons:</strong> 
            <ul>
                <li><code>set_goal_button</code>: Button to set a new goal.</li>
                <li><code>cancel_goal_button</code>: Button to cancel the current goal.</li>
            </ul>
        </li>
        <li><strong>Goal Status:</strong> <code>goal_status</code>: Displays the current status of the goal.</li>
        <li><strong>Plot Selector:</strong> <code>plot_selector</code>: Dropdown menu to select between position and velocity plots.</li>
    </ul>
    <p>All widgets are displayed using the <code>display</code> function and returned for further use.</p>
</div>

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px; margin-top: 20px;">
    <h3>Function: update_labels</h3>
    <ul>
        <li><strong>Update Position:</strong> The <code>pos_label</code> is updated to show the current position.</li>
        <li><strong>Update Velocity:</strong> The <code>vel_label</code> is updated to show the current velocity.</li>
    </ul>
</div>


In [ ]:
def init_widgets():
    global goal_status, plot_selector  
    pos_label = widgets.Label(value="Position: (0.0, 0.0)")
    vel_label = widgets.Label(value="Velocity: linear_x=0.0, angular_z=0.0")
    target_label = widgets.Label(value="Target: (0.0, 0.0)")    
    goal_x_input = widgets.FloatText(description="Goal X:")
    goal_y_input = widgets.FloatText(description="Goal Y:")
    set_goal_button = widgets.Button(description="Set New Goal")   
    goal_status = widgets.Text(value="No goal set", description='Goal Status:', disabled=True)
    cancel_goal_button = widgets.Button(description="Cancel Goal")        
    plot_selector = widgets.Dropdown(
    options=['Position Plot', 'Velocity Plot'],
    value='Position Plot',
    description='Select Plot:',
    )
    
    display(pos_label, vel_label, target_label, goal_x_input, goal_y_input, set_goal_button, cancel_goal_button, goal_status, plot_selector) #modified
    return pos_label, vel_label, target_label, goal_x_input, goal_y_input, set_goal_button, cancel_goal_button, goal_status , plot_selector #modified 

def update_labels(pos_x, pos_y, vel_x, vel_z, pos_label, vel_label):
    pos_label.value = f"Position: ({pos_x}, {pos_y})"
    vel_label.value = f"Velocity: linear_x={vel_x}, angular_z={vel_z}"


# Publishing Position and Velocity

The `publish_position_velocity` function processes the incoming odometry messages to extract the robot's current position and velocity, updates the corresponding labels, and publishes this information.

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px;">
    <h3>Function: publish_position_velocity</h3>
    <ul>
        <li><strong>Global Variables:</strong> The function uses global variables <code>pos_label</code>, <code>vel_label</code>, and <code>pub</code> to update labels and publish messages.</li>
        <li><strong>Extract Position:</strong> 
            <ul>
                <li>The current position of the robot is extracted from the odometry message using <code>msg.pose.pose.position</code>.</li>
            </ul>
        </li>
        <li><strong>Extract Velocity:</strong> 
            <ul>
                <li>The current linear velocity of the robot is extracted using <code>msg.twist.twist.linear</code>.</li>
                <li>The current angular velocity of the robot is extracted using <code>msg.twist.twist.angular</code>.</li>
            </ul>
        </li>
        <li><strong>Create Vel Message:</strong> 
            <ul>
                <li>A new <code>Vel</code> message is created to hold the extracted position and velocity data.</li>
                <li>Position data is assigned to <code>pos_and_vel.pos_x</code> and <code>pos_and_vel.pos_y</code>.</li>
                <li>Velocity data is assigned to <code>pos_and_vel.vel_x</code> and <code>pos_and_vel.vel_z</code>.</li>
            </ul>
        </li>
        <li><strong>Publish Message:</strong> The <code>Vel</code> message containing the current position and velocity is published using <code>pub.publish(pos_and_vel)</code>.</li>
        <li><strong>Update Labels:</strong> The labels for position and velocity are updated using the <code>update_labels</code> function.</li>
    </ul>
</div>


In [ ]:
def publish_position_velocity(msg):
    global pos_label ,vel_label
    global pub
    current_pos = msg.pose.pose.position
    current_vel_linear = msg.twist.twist.linear
    current_vel_angular = msg.twist.twist.angular   
    pos_and_vel = Vel()
    pos_and_vel.pos_x = current_pos.x
    pos_and_vel.pos_y = current_pos.y
    pos_and_vel.vel_x = current_vel_linear.x
    pos_and_vel.vel_z = current_vel_angular.z    
    pub.publish(pos_and_vel)   
    update_labels(current_pos.x, current_pos.y, current_vel_linear.x, current_vel_angular.z, pos_label, vel_label)

# Visualizing Robot Position and Velocity

The `Visualiser` class is responsible for dynamically visualizing the robot's position and velocity on a single plot. The class updates the plot based on the selected plot type (position or velocity) and handles the incoming odometry messages.

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px;">
    <h3>Class: Visualiser</h3>
    <ul>
        <li><strong>Initialization (<code>__init__</code>):</strong>
            <ul>
                <li>Creates a figure and axis for the plot using <code>plt.subplots()</code>.</li>
                <li>Initializes two line objects for position (blue circles) and velocity (red line) plots.</li>
                <li>Initializes empty lists <code>x_data</code>, <code>y_data</code>, and <code>vel_data</code> to store data points.</li>
                <li>Sets the default plot type to position plot with <code>is_position_plot = True</code>.</li>
            </ul>
        </li>
        <li><strong>Plot Initialization (<code>plot_init</code>):</strong>
            <ul>
                <li>Sets the x and y axis limits to -10 and 10.</li>
                <li>Adds grid lines to the plot.</li>
                <li>Makes both position and velocity lines visible initially.</li>
                <li>Adds a legend to the plot.</li>
                <li>Returns the position and velocity line objects.</li>
            </ul>
        </li>
        <li><strong>Odometry Callback (<code>odom_callback</code>):</strong>
            <ul>
                <li>Appends the robot's current position and linear velocity from the odometry message to <code>y_data</code>, <code>x_data</code>, and <code>vel_data</code> respectively.</li>
                <li>Limits the size of <code>vel_data</code> to 100 data points.</li>
            </ul>
        </li>
        <li><strong>Update Plot (<code>update_plot</code>):</strong>
            <ul>
                <li>Updates the position plot with the current position data if <code>is_position_plot</code> is <code>True</code>.</li>
                <li>Updates the velocity plot with the current velocity data if <code>is_position_plot</code> is <code>False</code>.</li>
                <li>Adjusts the x and y axis limits accordingly.</li>
                <li>Returns the position and velocity line objects.</li>
            </ul>
        </li>
        <li><strong>Switch Plot (<code>switch_plot</code>):</strong>
            <ul>
                <li>Sets the <code>is_position_plot</code> flag based on the selected plot type.</li>
                <li>Updates the plot visibility, axis limits, labels, and title based on the selected plot type.</li>
                <li>Redraws the plot using <code>plt.draw()</code>.</li>
            </ul>
        </li>
    </ul>
</div>


In [ ]:
%matplotlib notebook

class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.position_ln, = plt.plot([], [], 'bo', label='Position')
        self.velocity_ln, = plt.plot([], [], 'r', label='Velocity')
        self.x_data, self.y_data = [], []
        self.vel_data = []
        self.is_position_plot = True  

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        self.ax.grid(True)  
        self.position_ln.set_visible(True)
        self.velocity_ln.set_visible(True)
        self.ax.legend()
        return self.position_ln, self.velocity_ln
        
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        self.vel_data.append(msg.twist.twist.linear.x)
        
        if len(self.vel_data) > 100:
            self.vel_data.pop(0)

    def update_plot(self, frame):
        if self.is_position_plot:
            self.position_ln.set_data(self.x_data, self.y_data)
            self.position_ln.set_visible(True)
            self.velocity_ln.set_visible(False)
        else:
            self.velocity_ln.set_data(range(len(self.vel_data)), self.vel_data)
            self.position_ln.set_visible(False)
            self.velocity_ln.set_visible(True)
            self.ax.set_xlim(0, len(self.vel_data))
            self.ax.set_ylim(-1, 1)
        return self.position_ln, self.velocity_ln

    def switch_plot(self, plot_type):
        self.is_position_plot = (plot_type == 'Position Plot')
        if self.is_position_plot:
            self.position_ln.set_visible(True)
            self.velocity_ln.set_visible(False)
            self.ax.set_xlim(-10, 10)
            self.ax.set_ylim(-10, 10)
            self.ax.set_xlabel('X')
            self.ax.set_ylabel('Y')
            self.ax.set_title("Position Plot")  
        else:
            self.position_ln.set_visible(False)
            self.velocity_ln.set_visible(True)
            self.ax.set_xlim(0, len(self.vel_data))
            self.ax.set_ylim(-1, 1)
            self.ax.set_xlabel('Time')
            self.ax.set_ylabel('Velocity')
            self.ax.set_title("Velocity Plot") 
            
        plt.draw()


# Handling Plot Selection Change

The `on_plot_selector_change` function updates the plot displayed based on the user's selection from the dropdown menu. This function is triggered whenever the value of the dropdown menu changes.

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px;">
    <h3>Function: on_plot_selector_change</h3>
    <ul>
        <li><strong>Parameter:</strong> 
            <ul>
                <li><code>change</code>: A dictionary containing information about the change event. Specifically, the new value selected from the dropdown menu is accessed using <code>change['new']</code>.</li>
            </ul>
        </li>
        <li><strong>Switch Plot:</strong> 
            <ul>
                <li>The function calls <code>vis.switch_plot(change['new'])</code> to switch the plot displayed based on the new value selected from the dropdown menu.</li>
            </ul>
        </li>
    </ul>
</div>


In [ ]:
def on_plot_selector_change(change):
    vis.switch_plot(change['new'])

# Setting and Canceling Goals

The `set_new_goal` and `cancel_goal` functions manage the robot's movement goals. These functions allow the user to set new goals and cancel existing ones, updating the user interface accordingly.

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px;">
    <h3>Function: set_new_goal</h3>
    <ul>
        <li><strong>Global Variables:</strong> The function uses global variables <code>goal_x_input</code>, <code>goal_y_input</code>, <code>goal_status</code>, <code>goal_cancelled</code>, and <code>target_label</code> for input and status updates.</li>
        <li><strong>Read Input:</strong> 
            <ul>
                <li>Reads the x-coordinate and y-coordinate for the new goal from the input fields <code>goal_x_input</code> and <code>goal_y_input</code>.</li>
            </ul>
        </li>
        <li><strong>Set Parameters:</strong> 
            <ul>
                <li>Sets the ROS parameters <code>/des_pos_x</code> and <code>/des_pos_y</code> to the new goal coordinates using <code>rospy.set_param</code>.</li>
            </ul>
        </li>
        <li><strong>Create Goal:</strong> 
            <ul>
                <li>Creates a new <code>PlanningGoal</code> message with the target coordinates.</li>
            </ul>
        </li>
        <li><strong>Send Goal:</strong> 
            <ul>
                <li>Sends the goal to the action server using <code>client.send_goal</code>, with callbacks for done and feedback.</li>
            </ul>
        </li>
        <li><strong>Update Status:</strong> 
            <ul>
                <li>Sets <code>goal_cancelled</code> to <code>False</code>.</li>
                <li>Updates the target label to display the new target coordinates.</li>
                <li>Updates the goal status label to indicate the new goal coordinates.</li>
            </ul>
        </li>
    </ul>
</div>

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px; margin-top: 20px;">
    <h3>Function: cancel_goal</h3>
    <ul>
        <li><strong>Global Variables:</strong> The function uses global variables <code>goal_cancelled</code>, <code>goal_status</code>, and <code>target_label</code> for status updates.</li>
        <li><strong>Cancel Goal:</strong> 
            <ul>
                <li>Checks if a goal is currently active by verifying <code>goal_cancelled</code> is <code>False</code>.</li>
                <li>Cancels the current goal using <code>client.cancel_goal</code> and sets <code>goal_cancelled</code> to <code>True</code>.</li>
                <li>Logs the cancellation of the current goal using <code>rospy.loginfo</code>.</li>
                <li>Updates the goal status label to indicate the goal has been canceled.</li>
            </ul>
        </li>
        <li><strong>Update Target Label:</strong> 
            <ul>
                <li>Resets the target label to <code>(0.0, 0.0)</code> indicating no active goal.</li>
            </ul>
        </li>
        <li><strong>No Active Goal:</strong> 
            <ul>
                <li>If no active goal exists, logs a message indicating that there is no goal to cancel using <code>rospy.loginfo</code>.</li>
            </ul>
        </li>
    </ul>
</div>


In [ ]:
def set_new_goal(b):
    global goal_x_input , goal_y_input, goal_status, goal_cancelled 
    global target_label
    global goal_cancelled_flag
    input_x = goal_x_input.value
    input_y = goal_y_input.value
    rospy.set_param('/des_pos_x', input_x)
    rospy.set_param('/des_pos_y', input_y)
    goal = PlanningGoal()
    goal.target_pose.pose.position.x = input_x
    goal.target_pose.pose.position.y = input_y
    client.send_goal(goal, done_cb=goal_done_cb, feedback_cb=goal_feedback_cb) 
    goal_cancelled = False
    target_label.value = f"Target: ({input_x}, {input_y})"
    goal_status.value = f"Goal set at ({input_x}, {input_y}, 0)"  
    
def cancel_goal(b):
    global goal_cancelled, goal_status  
    global target_label
    if not goal_cancelled:
        goal_cancelled = True
        client.cancel_goal()
        rospy.loginfo("Current goal has been cancelled")
        goal_status.value = "Goal canceled"  
    else:
        rospy.loginfo("No active goal to cancel")
    
    target_label.value = "Target: (0.0, 0.0)"


# Handling Goal Results and Feedback

The `goal_done_cb` and `goal_feedback_cb` functions manage the completion and feedback of goals, updating the user interface and performance metrics accordingly.

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px;">
    <h3>Function: goal_done_cb</h3>
    <ul>
        <li><strong>Global Variables:</strong> The function uses global variables <code>target_reached</code>, <code>target_canceled</code>, and <code>goal_status</code> to update the status and performance metrics.</li>
        <li><strong>Parameters:</strong>
            <ul>
                <li><code>state</code>: The final state of the goal, indicating whether it was successful or not.</li>
                <li><code>result</code>: The result returned by the action server after goal completion.</li>
            </ul>
        </li>
        <li><strong>Check Goal Status:</strong> 
            <ul>
                <li>If the goal state is <code>SUCCEEDED</code>, increment the <code>target_reached</code> counter and update the goal status to "Goal reached".</li>
                <li>If the goal did not succeed, increment the <code>target_canceled</code> counter and update the goal status to "Goal canceled".</li>
            </ul>
        </li>
        <li><strong>Update Performance Metrics:</strong> 
            <ul>
                <li>Calls <code>vix_goals.update(target_reached, target_canceled)</code> to update the bar chart displaying the number of successful and canceled goals.</li>
            </ul>
        </li>
    </ul>
</div>

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px; margin-top: 20px;">
    <h3>Function: goal_feedback_cb</h3>
    <ul>
        <li><strong>Parameter:</strong>
            <ul>
                <li><code>feedback</code>: Feedback information sent by the action server while the goal is being processed.</li>
            </ul>
        </li>
        <li><strong>Functionality:</strong> 
            <ul>
                <li>The function is currently a placeholder and does not perform any operations. It can be extended to handle real-time feedback from the action server if needed.</li>
            </ul>
        </li>
    </ul>
</div>


In [ ]:
target_reached = 0
target_canceled = 0

def goal_done_cb(state, result):
    global target_reached, target_canceled, goal_status  
    if state == actionlib.GoalStatus.SUCCEEDED:
        target_reached += 1
        goal_status.value = "Goal reached"
    else:
        target_canceled += 1
        goal_status.value = "Goal canceled"
    vix_goals.update(target_reached, target_canceled)  

def goal_feedback_cb(feedback):
    pass


# Visualizing Goal Status

The `VisualizerGoals` class is responsible for creating a bar chart to visualize the number of successful and failed goals. The chart updates dynamically based on the latest goal performance metrics.

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px;">
    <h3>Class: VisualizerGoals</h3>
    <ul>
        <li><strong>Initialization (<code>__init__</code>):</strong>
            <ul>
                <li>Creates a figure and axis for the bar chart using <code>plt.subplots()</code>.</li>
                <li>Initializes the categories for the bar chart as <code>['Successful', 'Failed']</code>.</li>
                <li>Adds grid lines to the chart using <code>self.ax.grid(True)</code>.</li>
                <li>Initializes the count of goals as a list <code>self.n_goals = [0, 0]</code>.</li>
                <li>Sets the colors for the bars using <code>self.bar_colors = ['tab:green', 'tab:red']</code>.</li>
                <li>Creates the bar chart with initial values using <code>self.ax.bar(self.res, self.n_goals, color=self.bar_colors)</code>.</li>
                <li>Sets the title of the chart to 'Counting the State of the Goals - Successful or Failed'.</li>
                <li>Sets the y-axis limit to 10 using <code>self.ax.set_ylim(0, 10)</code>.</li>
            </ul>
        </li>
        <li><strong>Update Chart (<code>update</code>):</strong>
            <ul>
                <li>Updates the count of successful and failed goals using <code>self.n_goals = [s, c]</code>, where <code>s</code> is the number of successful goals and <code>c</code> is the number of failed goals.</li>
                <li>Sets the height of each bar in the chart to reflect the updated counts using <code>for bar, height in zip(self.bar, self.n_goals): bar.set_height(height)</code>.</li>
                <li>Recomputes the limits of the axes using <code>self.ax.relim()</code> and updates the view limits using <code>self.ax.autoscale_view()</code>.</li>
                <li>Redraws the chart canvas using <code>self.fig.canvas.draw()</code>.</li>
            </ul>
        </li>
    </ul>
</div>


In [ ]:
class VisualizerGoals:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.res = ['Successful', 'Failed']
        self.ax.grid(True)
        self.n_goals = [0, 0]
        self.bar_colors = ['tab:green', 'tab:red']
        self.bar = self.ax.bar(self.res, self.n_goals, color=self.bar_colors)
        self.ax.set_title('Counting the State of the Goals - Successful or Failed')
        self.ax.set_ylim(0, 10)

    def update(self, s, c):
        self.n_goals = [s, c]
        for bar, height in zip(self.bar, self.n_goals):
            bar.set_height(height)
        self.ax.relim()
        self.ax.autoscale_view()
        self.fig.canvas.draw()

# Main Function: Initializing and Running the System

The main function initializes all components, sets up ROS subscribers, and starts the interactive interface for managing and visualizing the robot's goals and movements.

<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px;">
    <h3>Main Function</h3>
    <ul>
        <li><strong>Initialize Widgets:</strong>
            <ul>
                <li>Calls <code>init_widgets()</code> to initialize and display the user interface widgets.</li>
                <li>Unpacks the returned widget objects: <code>pos_label</code>, <code>vel_label</code>, <code>target_label</code>, <code>goal_x_input</code>, <code>goal_y_input</code>, <code>set_goal_button</code>, <code>cancel_goal_button</code>, <code>goal_status</code>, and <code>plot_selector</code>.</li>
            </ul>
        </li>
        <li><strong>Initialize ROS Components:</strong>
            <ul>
                <li>Calls <code>ros_components_init()</code> to initialize the ROS node, action client, and publisher.</li>
                <li>Unpacks the returned objects: <code>pub</code>, <code>client</code>, and <code>goal_cancelled</code>.</li>
            </ul>
        </li>
        <li><strong>Set Up ROS Subscriber:</strong>
            <ul>
                <li>Subscribes to the <code>/odom</code> topic to receive odometry messages using <code>rospy.Subscriber("/odom", Odometry, publish_position_velocity)</code>.</li>
            </ul>
        </li>
        <li><strong>Initialize Visualizers:</strong>
            <ul>
                <li>Creates an instance of the <code>Visualiser</code> class to handle the dynamic plot for position and velocity.</li>
                <li>Creates an instance of the <code>VisualizerGoals</code> class to handle the bar chart for goal performance.</li>
            </ul>
        </li>
        <li><strong>Set Up Odometry Subscriber:</strong>
            <ul>
                <li>Subscribes to the <code>/odom</code> topic again to receive odometry messages using <code>vis.odom_callback</code> to update the visualizer.</li>
            </ul>
        </li>
        <li><strong>Set Up Plot Animation:</strong>
            <ul>
                <li>Uses <code>FuncAnimation</code> from Matplotlib to update the plot dynamically with <code>ani1 = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)</code>.</li>
            </ul>
        </li>
        <li><strong>Set Up Plot Selector Observer:</strong>
            <ul>
                <li>Attaches the <code>on_plot_selector_change</code> function to the <code>plot_selector</code> dropdown menu to handle plot switching using <code>plot_selector.observe(on_plot_selector_change, names='value')</code>.</li>
                <li>Sets the default plot to 'Position Plot' using <code>on_plot_selector_change({'new': 'Position Plot'})</code>.</li>
            </ul>
        </li>
        <li><strong>Display Plot:</strong>
            <ul>
                <li>Displays the plot in the Jupyter Notebook using <code>plt.show(block=True)</code>.</li>
            </ul>
        </li>
        <li><strong>Attach Button Callbacks:</strong>
            <ul>
                <li>Attaches the <code>set_new_goal</code> function to the 'Set New Goal' button using <code>set_goal_button.on_click(set_new_goal)</code>.</li>
                <li>Attaches the <code>cancel_goal</code> function to the 'Cancel Goal' button using <code>cancel_goal_button.on_click(cancel_goal)</code>.</li>
            </ul>
        </li>
    </ul>
</div>


In [ ]:
if __name__ == "__main__":
    
    pos_label, vel_label, target_label, goal_x_input, goal_y_input, set_goal_button, cancel_goal_button, goal_status, plot_selector =  init_widgets()
    pub, client, goal_cancelled = ros_components_init()    
    rospy.Subscriber("/odom", Odometry, publish_position_velocity)
    vis = Visualiser()
    vix_goals = VisualizerGoals()      
    sub1 = rospy.Subscriber('/odom', Odometry, vis.odom_callback)    
    ani1 = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)    
    plot_selector.observe(on_plot_selector_change, names='value') 
    on_plot_selector_change({'new': 'Position Plot'})  

    plt.show(block=True)
    set_goal_button.on_click(set_new_goal)
    cancel_goal_button.on_click(cancel_goal)